In [ ]:
!pip install --upgrade pip
!pip install segtok
!pip install tiny-tokenizer
!pip install flair
!pip install pytorch_pretrained_bert

In [ ]:
from flair.data import Sentence
from flair.models.sequence_tagger_model import SequenceTagger

In [ ]:
import flair
import torch
import os
import sys

In [ ]:
flair.device = torch.device('cpu')
# 
tagger: SequenceTagger = SequenceTagger.load("ner")
print ("load protocol model")

#to be removed
sys.argv[1] = './inputtxt/'
sys.argv[2] = './output/'
outfolder = sys.argv[2]
for file in os.listdir(sys.argv[1]):
    if not file.endswith(".txt"):
        continue
    #print (file)
    all_entities = []
    offset = 0
    #print (sys.argv[1]+file) 
    for line in open(sys.argv[1]+file).readlines(): 
        print ("--------")
        print (line.strip())
        sentence: Sentence = Sentence(line.strip())
        print(dir(sentence))
        print(tagger.predict(sentence))        
        print (sentence.to_tagged_string())
        print(sentence.get_token(0))
        print(sentence.convert_tag_scheme())
        print(sentence.__dict__)
        print(dir(sentence))
        entities = sentence.to_offset_tags(offset)
        print (entities)
        new_entities = []
        for ent in entities:
            #if ent[3] not in ["this study", "that study", "the disease", "the finding", "the study", "the analysis", "treatment", "test", "testing", "treatment", "symptoms", "Lilly"]:
            new_entities.append(ent)
        offset += len(line)
        #print(sentence.to_tagged_string())
        all_entities += new_entities
        #print (entities)
    
    filew = open(outfolder + file.replace(".txt", ".ann"), "w")
    filew2 = open(outfolder + file, "w")
    txt = open(sys.argv[1]+file).read()
    filew2.write(txt)
    filew2.close
    index = 1
    for entity in all_entities: 
        fixed = False
        if txt[int(entity[1]):int(entity[2])] != entity[3]:
            for offset in [-5, -4, -3, -2, -1, 1, 2, 3,4,5]:
                if (offset < 0 and int(entity[1]) >= abs(offset) and txt[int(entity[1])-offset:int(entity[2])-offset] == entity[3]) or (offset > 0 and int(entity[2]) <= len(txt)-offset and txt[int(entity[1])+offset:int(entity[2])+offset] == entity[3]):
                    fixed = True
                    entity[1] = str(int(entity[1]) + offset)
                    entity[2] = str(int(entity[2]) + offset)
                    
            #if not fixed:
            #    print ("does not match found:")
            #    print (entity)
            #    print (txt[int(entity[1]):int(entity[2])] )
        else:
            fixed = True
        if fixed:
            filew.write("T"+str(index)+"\t"+entity[0]+" "+entity[1]+" "+entity[2]+"\t"+entity[3]+"\n")
        index += 1
    filew.close()